<table><tr>
<td> <img src="../../../../images/URV.png" style="width: 250px;"/> </td>
<td> <img src="../../../../images/UOC.jpg" style="width: 250px;"/> </td>
</tr></table>

# Creating models for predicting crypto returns

The objective of this notebook is to establish a model capable of predicting crypto price movements. For that, we will apply all the data seen up until now and use machine learning models for the forcasting. Those models will be:

* Logistic regression
* Naive Bayes
* Support vector classification
* K-Nearest Neighbor
* Decision Tree Classifier
* Random Forest Classifier

At first, we will use a 0 minutes delay, i.e., we will assume when a tweet is published, it has an immediate effect on the price movement. However, it is possible that from the time a tweet is issued until the market corrects may take some time, so we will include a delay variable for subsequent studies.

### Import libraries

In [1]:
import psycopg2
import datetime as dt
import time

from login_access import host, port, user, password

import numpy as np
import pandas as pd
pd.set_option('display.float_format', lambda x: '%.4f' % x)

from models_module import cont_train_test_X_y_split, min_max_scaler, discr_train_test_X_y_split, LogisticReg, RandomForestClas, DecisionTreeClas, SVC, KNN, NaiveBayes

import matplotlib.pyplot as plt
import seaborn as sns
%matplotlib inline

import sklearn.model_selection
import sklearn.linear_model
import sklearn.naive_bayes
import sklearn.svm
import sklearn.neighbors
import sklearn.tree
import sklearn.ensemble
from sklearn.metrics import classification_report

*** 

## Declare Variables

In [2]:
crypto = 'ADA' #Crypto symbol
delay = 0 #minutes

***

### Fetch data (part 1)

In [3]:
def round_to_next_minute(date, delay = 0):
    remainder = 60 - int(time.mktime(date.timetuple())) % 60 
    date_rounded = int(time.mktime(date.timetuple())) + remainder - delay*60
    #date_rounded_ = dt.datetime.utcfromtimestamp(date_rounded).strftime('%Y-%m-%d %H:%M:%S')
    return float(date_rounded)


def connect_to_db(symbol = None, delay = 0):
    # Connect to DB with credentials
    conn = psycopg2.connect(host=host, database="TwitterDB", port=port, user=user, password=password)

    # Create the cursor to execute SQL commands
    cur = conn.cursor()
    
    # Declare the SQL sentence
    if not symbol: command = "SELECT * FROM crypto_table;"
    else: command = "SELECT * FROM crypto_table WHERE symbol = '{}';".format(symbol)

    # Execute the SQL command
    cur.execute(command)

    # Save execution into a variable
    df = pd.DataFrame(cur.fetchall())
    df.columns=[ col.name for col in cur.description]
    
    # Let's then convert the datetime 'created_at' variable in a unix time variable 
    df['timestamp2'] = [time.mktime(x.timetuple()) for x in df['created_at']]
    
    # Then we pass the function 'round_to_next_minute' where we round to the next minute with 00 seconds
    ## e.g.: from 6h 43min 34sec to 6h 44mins 00 secs
    df['timestamp'] = [round_to_next_minute(x, delay = delay) for x in df['created_at']]
    
    # In order to have only ONE register per minute, in case of conflict we will choose the one 
    # that has the highest retweets
    unique_tweets = np.array(df.groupby('timestamp').max('retweet_count')['tweet_id'])
    df = df[df.tweet_id.isin(unique_tweets)]
    
    # Set that last variable to index for the joining process
    #df.set_index('timestamp', inplace = True)
    
    return df

### Fetch data (part 2)

In [4]:
def crypto_prices(symbol):
    path = '../../../../Crypto prices/'
    price_time_series = pd.read_csv(path + '{}USDT.csv'.format(symbol),
                                    sep='|',
                                    index_col=False,
                                    names=['timestamp', 'open', 'high', 'low', 'close', 'volume', 
                                           'tb_quote_a_v', 'tb_base_a_v','quote_a_v', 'trades']).\
                                    drop(['tb_quote_a_v', 'tb_base_a_v','quote_a_v'], axis = 1)
    
    price_time_series['daydate'] = [dt.datetime.utcfromtimestamp(x) for x in price_time_series['timestamp']]
    #price_time_series['daydate'] = [dt.datetime.fromtimestamp(0, datetime.timezone.utc)]
    
    max_date = '2021-12-31 23:59:59'
    max_date = dt.datetime.strptime(max_date, '%Y-%m-%d %H:%M:%S')
    
    price_time_series = price_time_series[price_time_series['daydate'] <= max_date]

    #price_time_series.set_index('timestamp', inplace = True)
    
    return price_time_series

### Unite data

In [5]:
def unite_tables(prices_table, twitter_table):
    df = pd.merge(prices_table, twitter_table, on='timestamp', how='left')
    df['sentiment_score']=np.float64(df['sentiment_score'])
    df['log_ret'] = np.log(df.close) - np.log(df.close.shift(1))
    df['bin_ret']= [1 if x > 0 else 0 for x in df.log_ret]
    df['dif_timestamp'] = df.timestamp - df.timestamp.shift(1)
    #df.retweet_count.fillna(0, inplace = True)
    df.sentiment_score.fillna(0, inplace = True)
    df.retweet_count.fillna(0, inplace = True)
    df.outcome.fillna('neutral', inplace = True)
    df = df[~df.log_ret.isna()].reset_index()
    return df

# What if we delayed the the returns _for all cryptos_?

In this scenario we will assume that the impact of tweets on prices have some sort of delay. We will assume 3 scenarios where 1, 2 and 5 minutes delays will be studied.

In [6]:
def scenario_creator(crypto = None, delay = 0):
    
    LRauc, NBauc, SVCauc, KNNauc, DTCauc, RFCauc, crypt = [], [], [], [], [], [], []
    
    for symbol in crypto:

        df_model = unite_tables(crypto_prices(symbol = symbol), connect_to_db(symbol = symbol, delay = delay))

        X_train, X_test, y_train, y_test = discr_train_test_X_y_split(df_model, features = ['sentiment_score'])

        X_train, X_test = min_max_scaler(X_train, X_test)

        crypt.append(symbol)

        LRauc.append(LogisticReg(X_train, y_train, X_test, y_test, random_state=123))

        NBauc.append(NaiveBayes(X_train, y_train, X_test, y_test))

        SVCauc.append(SVC(X_train, y_train, X_test, y_test))

        KNNauc.append(KNN(X_train, y_train, X_test, y_test))

        DTCauc.append(DecisionTreeClas(X_train, y_train, X_test, y_test, random_state=123))

        RFCauc.append(RandomForestClas(X_train, y_train, X_test, y_test, random_state=123))


    results = {'Crypto':crypt,
               'Log. Reg.': LRauc,
               'Naive Bayes': NBauc,
               'SVC': SVCauc,
               'KNN':KNNauc,
               'Decision tree': DTCauc,
               'Random Forest': RFCauc}

        
    return pd.DataFrame(results).set_index('Crypto')

In [7]:
crypto = ['BTC', 'ETH', 'BNB', 'LTC', 'ADA', 'XRP', 'LINK', 'MATIC', 
          'ATOM', 'DOGE', 'DOT', 'SOL', 'LUNA', 'AVAX', 'SHIB']

In [8]:
results_1 = scenario_creator(crypto = crypto, delay = 1)
plt.close()

In [9]:
results_1

,Log. Reg.,Naive Bayes,SVC,KNN,Decision tree,Random Forest
Crypto,,,,,,
BTC,0.4980,0.4980,0.5020,0.5000,0.5040,0.5040
ETH,0.4660,0.5340,0.4660,0.5010,0.5130,0.5110
BNB,0.5240,0.5240,0.5240,0.5280,0.5240,0.5150
LTC,0.5140,0.5070,0.5140,0.5590,0.5010,0.5300
ADA,0.5250,0.5080,0.5000,0.5320,0.4930,0.5140
XRP,0.5120,0.4840,0.5000,0.5020,0.4910,0.4990
LINK,0.4880,0.4850,0.5050,0.4770,0.4950,0.5270
MATIC,0.4680,0.4680,0.4680,0.4150,0.4580,0.3760
ATOM,0.5650,0.5650,0.4200,0.4290,0.5620,0.4830


In [10]:
results_1.to_csv('results_1')

In [11]:
results_2 = scenario_creator(crypto = crypto, delay = 2)
plt.close()

/Users/ricky/miniforge3/envs/env_tensorflow/lib/python3.9/site-packages/sklearn/svm/_base.py:1225: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  warnings.warn(


In [12]:
results_2

,Log. Reg.,Naive Bayes,SVC,KNN,Decision tree,Random Forest
Crypto,,,,,,
BTC,0.4970,0.4970,0.5000,0.5100,0.4930,0.4970
ETH,0.4940,0.4940,0.5060,0.5150,0.4990,0.5020
BNB,0.4980,0.4980,0.5020,0.5200,0.5030,0.5180
LTC,0.5290,0.4710,0.5290,0.4760,0.5010,0.4570
ADA,0.4660,0.5410,0.5340,0.5850,0.5030,0.5840
XRP,0.5100,0.5100,0.4720,0.5090,0.5120,0.5170
LINK,0.5250,0.4750,0.4750,0.5370,0.5060,0.5060
MATIC,0.5040,0.4960,0.4960,0.4250,0.5000,0.5400
ATOM,0.6080,0.4040,0.2900,0.6120,0.5000,0.4370


In [13]:
results_2.to_csv('results_2')

In [14]:
results_5 = scenario_creator(crypto = crypto, delay = 5)
plt.close()

In [15]:
results_5

,Log. Reg.,Naive Bayes,SVC,KNN,Decision tree,Random Forest
Crypto,,,,,,
BTC,0.4930,0.4950,0.4930,0.4990,0.5000,0.4960
ETH,0.4970,0.4970,0.4970,0.4700,0.5030,0.4970
BNB,0.5350,0.4650,0.4650,0.5090,0.5040,0.5020
LTC,0.4860,0.5070,0.5140,0.4920,0.5050,0.4900
ADA,0.4690,0.5000,0.4690,0.5070,0.5010,0.4870
XRP,0.4890,0.4920,0.4890,0.5060,0.5090,0.5040
LINK,0.4950,0.4950,0.5070,0.5260,0.5180,0.5250
MATIC,0.5220,0.4630,0.4780,0.5500,0.5030,0.4740
ATOM,0.4930,0.5650,0.4930,0.5570,0.5000,0.5310


In [16]:
results_5.to_csv('results_5')